In [1]:
import os
import cv2
import numpy as np
from torchvision.datasets import CIFAR10
from torchvision import transforms
from sklearn.cluster import MiniBatchKMeans
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score, classification_report
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim

import math

In [2]:
# --- Configuration ---
DATA_DIR = './data'
N_CLUSTERS = 5000    # BoVW 词典大小
MAX_KP = 30         # 每张图最大关键点数
PATCH_SIZE = 16     # 描述子 patch 大小
INTEREST_METHOD = 'DoG'  # 可选: 'DoG', 'Harris', 'LoG'
EPOCHS = 50
LR = 1e-3

In [3]:
# --- Setup MPS device ---
device = torch.device('mps') if torch.backends.mps.is_available() else torch.device('cpu')

In [8]:
# --- 1. Interest Point Detection ---
def detect_interest_points(gray, method='DoG', threshold=5):
    if method == 'DoG':
        # Define multiple sigma values for Gaussian blur
        num_octaves=3 # the level of the image pyramid
        scales_per_octave=3 # the number of scales per octave
        sigma0=1.6 # otiginal sigma value
        
        keypoints = []
        k = 2 ** (1 / scales_per_octave)  # 每层之间的尺度倍增因子
        
        # form the 0th octave to num_octaves-1 octaves
        for octave in range(num_octaves):
            # 1) create the base image for the current octave
            if octave == 0:
                base = gray.copy()
            else:
                # 下采样前一个 octave 的 base
                base = cv2.pyrDown(prev_base)
            
            # save the current base for the next octave
            prev_base = base
            # calculate the size of the base image
            h, w = base.shape
            
            # 2) 构建本 octave 的 Gaussian 金字塔
            #    需要做 (scales_per_octave + 3) 层，以便产生 scales_per_octave+2 个 DoG 层
            sigmas = [sigma0 * (k ** (i + octave*scales_per_octave)) for i in range(scales_per_octave + 3)]
            gaussians = [cv2.GaussianBlur(base, (0,0), sigmaX=s) for s in sigmas]
            
            # 3) 计算 DoG 图层
            dogs = []
            for i in range(len(gaussians)-1):
                diff = gaussians[i+1].astype(np.float32) - gaussians[i].astype(np.float32)
                dogs.append(diff)
            
            # 4) 在空间 + 尺度域寻找极值点
            #    只从第 1 层到倒数第 2 层做比较，因为要用前后层做上下尺度比较
            for i in range(1, len(dogs)-1):
                # the three layers to compare
                prev_d, curr_d, next_d = dogs[i-1], dogs[i], dogs[i+1]

                # max_spat and min_spat are masks of the same size as curr_d
                # 空间极值：用形态学膨胀/腐蚀检测局部极大/极小值
                # 对 curr_d 做形态学膨胀（dilation），相当于每个像素点被它 3×3 邻域内的最大值取代。
                #curr_d 在每个像素的 3×3 邻域内取最大值，输出也是一个 (H, W) 的数组——max_spat[y, x] 就是 curr_d 在 (y, x) 周围那 9 个点里的最大值。
                max_spat = cv2.dilate(curr_d, np.ones((3,3),np.uint8))
                # 做腐蚀（erosion），相当于每个像素点被邻域内的最小值取代。
                min_spat = cv2.erode( curr_d, np.ones((3,3),np.uint8))
                
                # 跨尺度极值：要比前一 DoG 和后一 DoG 都大（或都小）
                mask_max = (curr_d == max_spat) & (curr_d > prev_d) & (curr_d > next_d) & (np.abs(curr_d) > threshold)
                mask_min = (curr_d == min_spat) & (curr_d < prev_d) & (curr_d < next_d) & (np.abs(curr_d) > threshold)
                
                # Find all coordinates of the points that satisfy the conditions
                coords = np.vstack((np.argwhere(mask_max), np.argwhere(mask_min)))

                # 5) 将坐标映射回原图尺度并创建 KeyPoint
                for y, x in coords:
                    # 因为后面可能 downsample 了，需要乘回 2**octave
                    scale = (2 ** octave)

                    kp = cv2.KeyPoint(float(x*scale), float(y*scale), size=PATCH_SIZE * scale)
                    keypoints.append(kp)
                    
        # 最后按响应或数量截断
        keypoints = sorted(keypoints, key=lambda kp: kp.response if hasattr(kp, 'response') else 1.0,
                        reverse=True)
        return keypoints[:MAX_KP]
    elif method == 'Harris':
        harris = cv2.cornerHarris(gray.astype(np.float32), 2, 3, 0.04)
        coords = np.argwhere(harris > threshold * harris.max())
    elif method == 'LoG':
        log = cv2.Laplacian(gray, cv2.CV_64F)
        coords = np.argwhere(np.abs(log) > threshold)
    else:
        sift = cv2.SIFT_create()
        kps = open_kps = sift.detect(gray, None)
        # raise ValueError('Unsupported method')
    # Create keypoints and limit count
    return kps[:MAX_KP]

In [15]:
# --- 2. Compute Custom SIFT-like Descriptor ---
def compute_custom_sift_descriptor(gray, keypoints, patch_size=PATCH_SIZE):
    # Compute gradients once
    grad_x = cv2.Sobel(gray, cv2.CV_32F, 1, 0, ksize=3)
    grad_y = cv2.Sobel(gray, cv2.CV_32F, 0, 1, ksize=3)
    mag = cv2.magnitude(grad_x, grad_y)
    ori = cv2.phase(grad_x, grad_y, angleInDegrees=True)

    descriptors = []
    half = patch_size // 2
    h, w = gray.shape

    for kp in keypoints:
        x, y = int(kp.pt[0]), int(kp.pt[1])
        # boundary check
        if x-half < 0 or y-half < 0 or x+half >= w or y+half >= h:
            continue
        # extract full patch
        mag_full = mag[y-half:y+half, x-half:x+half]
        ori_full = ori[y-half:y+half, x-half:x+half]
        # 1) Orientation assignment: build 36-bin histogram over full patch
        hist8, bin_edges = np.histogram(
            ori_full.ravel(), bins=8, range=(0,360), weights=mag_full.ravel()
        )
        # main orientation is center of max bin
        max_idx = np.argmax(hist8)
        main_ori = (bin_edges[max_idx] + bin_edges[max_idx+1]) / 2.0
        # normalize full patch orientations relative to main orientation
        ori_full = (ori_full - main_ori + 360) % 360

        # slice into subregions and build descriptor
        desc = []
        for i in range(4):
            for j in range(4):
                sub_mag = mag_full[i*4:(i+1)*4, j*4:(j+1)*4].ravel()
                sub_ori = ori_full[i*4:(i+1)*4, j*4:(j+1)*4].ravel()
                # 2) histogram on aligned orientations
                hsub, _ = np.histogram(sub_ori, bins=8, range=(0,360), weights=sub_mag)
                desc.extend(hsub)
        desc = np.array(desc, dtype=np.float32)
        desc /= (np.linalg.norm(desc) + 1e-7)
        descriptors.append(desc)

    return np.array(descriptors) if descriptors else None

In [6]:
# --- 3. Load CIFAR-10 ---
transform = transforms.Compose([transforms.ToTensor()])
trainset = CIFAR10(DATA_DIR, train=True, download=True, transform=transform)
testset  = CIFAR10(DATA_DIR, train=False, download=True, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [16]:
# --- 4. Extract descriptors from training set ---
all_desc = []
img_desc_idx = []
train_labels = []
print('Extracting training descriptors...')
for img, label in tqdm(trainset, desc='Train images'):
    img_np = (img.numpy().transpose(1,2,0) * 255).astype(np.uint8)
    gray = cv2.cvtColor(img_np, cv2.COLOR_RGB2GRAY)
    kps = detect_interest_points(gray, method=INTEREST_METHOD)
    des = compute_custom_sift_descriptor(gray, kps)
    if des is not None:
        all_desc.extend(des)
    img_desc_idx.append(len(all_desc))
    train_labels.append(label)
all_desc = np.vstack(all_desc)

Extracting training descriptors...


Train images: 100%|██████████| 50000/50000 [04:46<00:00, 174.60it/s]


In [17]:
# --- 5. Build visual vocabulary ---
kmeans = MiniBatchKMeans(n_clusters=N_CLUSTERS, batch_size=1000, verbose=1)
kmeans.fit(all_desc)

Init 1/1 with method k-means++
Inertia for init 1/1: 8237.5478515625
[MiniBatchKMeans] Reassigning 500 cluster centers.
Minibatch step 1/37078: mean batch inertia: 0.5404362182617187
[MiniBatchKMeans] Reassigning 500 cluster centers.
Minibatch step 2/37078: mean batch inertia: 0.5494646606445313, ewa inertia: 0.5494646606445313
[MiniBatchKMeans] Reassigning 500 cluster centers.
Minibatch step 3/37078: mean batch inertia: 0.5187398681640625, ewa inertia: 0.5492989336389082
[MiniBatchKMeans] Reassigning 500 cluster centers.
Minibatch step 4/37078: mean batch inertia: 0.5167269897460938, ewa inertia: 0.5491232432503692
[MiniBatchKMeans] Reassigning 500 cluster centers.
Minibatch step 5/37078: mean batch inertia: 0.49302517700195314, ewa inertia: 0.5488206548912615
[MiniBatchKMeans] Reassigning 500 cluster centers.
Minibatch step 6/37078: mean batch inertia: 0.4800557861328125, ewa inertia: 0.5484497428403944
[MiniBatchKMeans] Reassigning 500 cluster centers.
Minibatch step 7/37078: mean b

MiniBatchKMeans(batch_size=1000, n_clusters=5000, verbose=1)

In [18]:
# --- 6. Build BoVW features for train ---
def build_bovw(descriptors):
    words = kmeans.predict(descriptors)
    hist, _ = np.histogram(words, bins=np.arange(N_CLUSTERS+1))
    return hist.astype(float) / (hist.sum() + 1e-7)

train_feats = []
start = 0
# for idx in img_desc_idx:
for idx in tqdm(img_desc_idx, desc='Building BoVW features'):
    end = idx
    if end - start > 0:
        train_feats.append(build_bovw(all_desc[start:end]))
    else:
        train_feats.append(np.zeros(N_CLUSTERS, dtype=float))
    start = end
train_feats = np.array(train_feats)
train_labels = np.array(train_labels)

Building BoVW features: 100%|██████████| 50000/50000 [00:20<00:00, 2392.32it/s]


In [19]:
# --- 7. Extract BoVW features for test set ---
test_feats = []
test_labels = []
print('Extracting test BoVW features...')
for img, label in tqdm(testset, desc='Test images'):
    img_np = (img.numpy().transpose(1,2,0) * 255).astype(np.uint8)
    gray = cv2.cvtColor(img_np, cv2.COLOR_RGB2GRAY)
    kps = detect_interest_points(gray, method=INTEREST_METHOD)
    des = compute_custom_sift_descriptor(gray, kps)
    if des is not None:
        hist = build_bovw(des)
    else:
        hist = np.zeros(N_CLUSTERS, dtype=float)
    test_feats.append(hist)
    test_labels.append(label)

test_feats = np.array(test_feats)
test_labels = np.array(test_labels)

Extracting test BoVW features...


Test images: 100%|██████████| 10000/10000 [01:06<00:00, 149.55it/s]


## Softmax

In [20]:
# --- 8. Train Softmax Regression (Logistic Regression) on MPS ---
# Prepare PyTorch tensors
X_train = torch.from_numpy(train_feats).float().to(device)
y_train = torch.from_numpy(train_labels).long().to(device)
X_test  = torch.from_numpy(test_feats).float().to(device)
y_test  = torch.from_numpy(test_labels).long().to(device)

# Define simple linear model
model = nn.Linear(N_CLUSTERS, 10).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LR, weight_decay=1e-4)

print('Training Softmax Regression...')
for ep in range(1, EPOCHS+1):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()
    if ep % 5 == 0:
        print(f'Epoch {ep}/{EPOCHS}, loss={loss.item():.4f}')

Training Softmax Regression...
Epoch 5/50, loss=2.3000
Epoch 10/50, loss=2.2969
Epoch 15/50, loss=2.2938
Epoch 20/50, loss=2.2907
Epoch 25/50, loss=2.2877
Epoch 30/50, loss=2.2847
Epoch 35/50, loss=2.2818
Epoch 40/50, loss=2.2790
Epoch 45/50, loss=2.2762
Epoch 50/50, loss=2.2735


In [21]:
# --- 9. Evaluate on test set ---
model.eval()
with torch.no_grad():
    preds = model(X_test).argmax(dim=1)
    acc = (preds == y_test).float().mean().item()
print(f'Test Accuracy: {acc:.4f}')
# Convert MPS tensor to CPU numpy for classification_report
preds_np = preds.cpu().numpy()
print(classification_report(test_labels, preds_np))

Test Accuracy: 0.2840
              precision    recall  f1-score   support

           0       0.27      0.36      0.31      1000
           1       0.32      0.34      0.33      1000
           2       0.28      0.23      0.26      1000
           3       0.21      0.17      0.19      1000
           4       0.22      0.15      0.18      1000
           5       0.25      0.28      0.26      1000
           6       0.27      0.30      0.28      1000
           7       0.36      0.35      0.36      1000
           8       0.33      0.40      0.36      1000
           9       0.31      0.25      0.28      1000

    accuracy                           0.28     10000
   macro avg       0.28      0.28      0.28     10000
weighted avg       0.28      0.28      0.28     10000



## SVM

In [ ]:
# 定义 SVM 管道并开启 verbose 打印收敛信息，关闭 shrinking 提速
svm_clf = make_pipeline(
    StandardScaler(),
    SVC(kernel='linear', probability=True, verbose=True, shrinking=False)
)

# 在 train_feats / train_labels 上训练
svm_clf.fit(train_feats, train_labels)
print("SVM training complete.")

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# 1. 预测
svm_preds = svm_clf.predict(test_feats)

# 2. 整体准确率
acc = accuracy_score(test_labels, svm_preds)
print(f"SVM Test Accuracy: {acc:.4f}")

# 3. 详细报告
print(classification_report(test_labels, svm_preds))

# 4. （可选）混淆矩阵
cm = confusion_matrix(test_labels, svm_preds)
print("Confusion Matrix:")
print(cm)